In [ ]:
import sys
sys.path.extend([ '../', '../../'])
import torch
import dataloader as dl
from args import Args
from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from train2 import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

In [ ]:
data = torch.load('../../data/aminer/am_data.pt')
print(data)

In [ ]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

In [ ]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

In [ ]:
encoder1 = Het_ConEn(args.embed_dim, args.dropout)
encoder2 = Het_NetEn(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
# decoder_a = torch.load('pretrained/decoder_a.pth')
# decoder_p = torch.load('pretrained/decoder_p.pth')
# decoder_v = torch.load('pretrained/decoder_v.pth')
decoder_list = [decoder_a, decoder_p, decoder_v]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [ ]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'preO')

In [ ]:
# torch.save(decoder_list[0], 'pretrained/decoder_a.pth')
# torch.save(decoder_list[1], 'pretrained/decoder_p.pth')
# torch.save(decoder_list[2], 'pretrained/decoder_v.pth')
# torch.save(encoder1, 'pretrained/encoder1.pth')
# torch.save(encoder2, 'pretrained/encoder2.pth')